In [3]:
import cx_Oracle
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import font_manager, rc
font_path = "C:/Windows/Fonts/NGULIM.TTF"
font = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font)
con = cx_Oracle.connect('LM_PDB/LM@localhost:1521/xepdb1')
cursor = con.cursor()

In [5]:
cursor.execute("select 고객번호, 대중분류코드,count(*),sum(구매금액) from purprod group by 고객번호, 대중분류코드 order by 고객번호")
x = cursor.fetchall()
df2 = pd.DataFrame(x,columns = ['고객번호','대중분류코드','구매횟수','총구매금액'])
df2

,고객번호,대중분류코드,구매횟수,총구매금액
0,00001,A0101,12,136310
1,00001,A0102,11,171060
2,00001,A0103,45,282810
3,00001,A0104,161,2122070
4,00001,A0105,35,811350
...,...,...,...,...
3198952,19383,D0601,10,18280
3198953,19383,D0801,3,80000
3198954,19383,D0802,19,191530
3198955,19383,D0803,2,16000


In [6]:
df22 = df2[['고객번호','대중분류코드','구매횟수']]
df22

,고객번호,대중분류코드,구매횟수
0,00001,A0101,12
1,00001,A0102,11
2,00001,A0103,45
3,00001,A0104,161
4,00001,A0105,35
...,...,...,...
3198952,19383,D0601,10
3198953,19383,D0801,3
3198954,19383,D0802,19
3198955,19383,D0803,2


In [26]:
cursor.execute("select 대중분류코드, 중분류명, count(*) from prodcl group by 대중분류코드, 중분류명")
x = cursor.fetchall()
df1 = pd.DataFrame(x,columns = ['대중분류코드','중분류명', '갯수'])
df1

,대중분류코드,중분류명,갯수
0,B0502,뿌리채소,7
1,A0103,축산가공,6
2,A0106,가공식품,55
3,A0110,차/커피,4
4,A0203,화장품,12
...,...,...,...
802,D0607,여행용품,1
803,D0702,남성,1
804,C0203,선식류,2
805,B6001,젓갈류,1


In [9]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split
from surprise import Reader
from surprise.model_selection import cross_validate
from surprise.model_selection import GridSearchCV

In [10]:
q1 = np.percentile(df22['구매횟수'],25)
q3 = np.percentile(df22['구매횟수'],75)
iqr = q3 - q1

df222 = df22.loc[(df22.구매횟수 >= q1 - 1.5*iqr) & (df22.구매횟수 <= q3 * 1.5*iqr)]
df222

,고객번호,대중분류코드,구매횟수
0,00001,A0101,12
1,00001,A0102,11
2,00001,A0103,45
4,00001,A0105,35
6,00001,A0107,54
...,...,...,...
3198952,19383,D0601,10
3198953,19383,D0801,3
3198954,19383,D0802,19
3198955,19383,D0803,2


In [11]:
df222.구매횟수.value_counts()

1     1044261
2      481069
3      290700
4      201374
5      151046
       ...   
81       1040
80       1040
84        979
83        971
82        970
Name: 구매횟수, Length: 84, dtype: int64

In [29]:
prods = df222.copy()
reader = Reader(rating_scale = (1,20))
data =Dataset.load_from_df(prods,reader)

In [30]:
algo = SVD(random_state = 0)
cross_validate(algo,data,measures=['RMSE','MAE'],cv = 5,verbose = True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    2.9843  2.9826  2.9896  2.9882  2.9867  2.9863  0.0025  
MAE (testset)     1.9282  1.9282  1.9336  1.9337  1.9319  1.9311  0.0025  
Fit time          219.09  219.84  220.15  219.08  219.42  219.51  0.42    
Test time         12.87   12.76   12.79   13.03   11.75   12.64   0.45    


{'test_rmse': array([2.9842966 , 2.98262818, 2.98959088, 2.98815037, 2.98674156]),
 'test_mae': array([1.92818567, 1.92815858, 1.93358882, 1.93370136, 1.93191467]),
 'fit_time': (219.0851972103119,
  219.83581495285034,
  220.14796996116638,
  219.07895421981812,
  219.42195200920105),
 'test_time': (12.869997024536133,
  12.755000114440918,
  12.793962001800537,
  13.029996871948242,
  11.750009059906006)}

In [12]:
df222.to_csv('df222.csv',index=False,header=False)

In [16]:
df2222 = pd.read_csv('df222.csv')

In [17]:
df2222

,00001,A0101,12
0,1,A0102,11
1,1,A0103,45
2,1,A0105,35
3,1,A0107,54
4,1,A0108,9
...,...,...,...
3154610,19383,D0601,10
3154611,19383,D0801,3
3154612,19383,D0802,19
3154613,19383,D0803,2


In [19]:
from surprise.dataset import DatasetAutoFolds

reader = Reader(line_format='user item rating',sep = ',',rating_scale = (1,100))
data_folds = DatasetAutoFolds(ratings_file='./df222.csv',reader=reader)
trainset = data_folds.build_full_trainset()

In [20]:
algo = SVD(n_epochs=20,n_factors=50,random_state = 0)
algo.fit(trainset)

In [21]:
uid = str('00008')
iid = str('D0802')
pred = algo.predict(uid,iid,verbose=True)

user: 00008      item: D0802      r_ui = None   est = 10.78   {'was_impossible': False}


In [27]:
def get_unbuy_surprise(ratings, prods, userId):
    buy_prods = ratings[ratings['고객번호']==userId]['대중분류코드'].tolist()
    total_prods = prods['대중분류코드'].tolist()
    
    unbuy_prods = [prod for prod in total_prods if prod not in buy_prods]
    print('구매한 상품 품목 수: ',len(buy_prods))
    print('추천대상 상품 품목 수: ',len(unbuy_prods))
    print('총 상품 품목 수: ',len(total_prods))
    
    return unbuy_prods

unbuy_prods = get_unbuy_surprise(df222,df1,'00001')

구매한 상품 품목 수:  128
추천대상 상품 품목 수:  679
총 상품 품목 수:  807


In [30]:
def recomm_prod_by_surprise(algo,userId,unbuy_prods,top_n=10):
    predictions = [algo.predict(str(userId),str(prodId)) for prodId in unbuy_prods]
    
    def sortkey_est(pred):
        return pred.est
    
    predictions.sort(key = sortkey_est, reverse=True)
    top_predictions= predictions[:top_n]
    
    top_prod_ids = [pred.iid for pred in top_predictions]
    top_prod_ratings = [pred.est for pred in top_predictions]
    
    top_prod_titles = df1[df1.대중분류코드.isin(top_prod_ids)]['중분류명']

    top_prod_preds = [(id , title, rating) for id , title, rating in
                      zip(top_prod_ids, top_prod_titles, top_prod_ratings)]
    
    return top_prod_preds

In [31]:
unbuy_prods = get_unbuy_surprise(df222,df1,'00001')
top_prod_preds = recomm_prod_by_surprise(algo, '00001', unbuy_prods, top_n = 10)

for top_prod in top_prod_preds:
    print(top_prod[1], ":", round(top_prod[2],2))

구매한 상품 품목 수:  128
추천대상 상품 품목 수:  679
총 상품 품목 수:  807
가공식품 : 40.73
농산물 : 36.11
애견용품 : 35.29
주유소 : 30.97
소주 : 27.52
기타음주류 : 25.74
음료 : 23.83
베이커리 : 23.63
문구/팬시 : 21.34
맥주 : 19.22
